# Doing Spatial viz in Python

In [ ]:
%load_ext autoreload
%autoreload 2

## Data: Weekly Payroll Jobs and Wages in Australia

Data provided by the Australian Bureau of Statistics pertaining to weekly numbers of jobs with payroll data. This is based on the Australian Tax Office's single touch payroll data, which is how most businesses report salaries and wages, pay as you go (PAYG) withholding, and superannuation.

[Weekly Jobs and Wages Index for Week ending 21/03/2021.](https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021#data-download)


[Excel file used -- Table 5: Sub-state - Payroll jobs indexes](https://www.abs.gov.au/statistics/labour/earnings-and-work-hours/weekly-payroll-jobs-and-wages-australia/week-ending-27-march-2021/6160055001_DO005.xlsx)

In [ ]:
import pandas as pd

# note this requires the openpyxl package
df = pd.read_excel("data/6160055001_DO005.xlsx", sheet_name=1)
df.head()

In [ ]:
import dtale

dtale.show(df)

In [ ]:
from utils.aus_jobs import clean_and_load_jobs_data

jobs_df = clean_and_load_jobs_data("data/6160055001_DO005.xlsx")
jobs_df.head()

In [ ]:
states_jobs = jobs_df.groupby(["STE_NAME16", "Date"])["Index"].mean()
states_jobs_df = states_jobs.reset_index()
country_jobs_df = states_jobs.mean(level="Date").reset_index()

In [ ]:
country_jobs_df

In [ ]:
states_jobs_df

In [ ]:
import plotly.express as px

px.line(
    country_jobs_df,
    x="Date",
    y="Index",
    title="Weekly Payroll Jobs and Wages Index",
    width=1200,
    height=500,
)

In [ ]:
px.line(
    states_jobs_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    width=1200,
    height=500,
)

In [ ]:
states_and_country_df = pd.concat(
    [states_jobs_df, country_jobs_df.assign(STE_NAME16="AUS")]
)
state_names = list(states_jobs_df["STE_NAME16"].unique())

px.line(
    states_and_country_df,
    x="Date",
    y="Index",
    color="STE_NAME16",
    title="Weekly Payroll Jobs and Wages Index by State",
    color_discrete_map={"AUS": "black"},
    category_orders={"STE_NAME16": ["AUS"] + state_names},
    line_dash="STE_NAME16",
    line_dash_sequence=["dot"] + ["solid" for _state in state_names],
    width=1200,
    height=500,
)

## Spatial Visualisations

For this section, w'll need the Australian, Statistical Area Level 4 shape file, from [here](https://www.abs.gov.au/AUSSTATS/subscriber.nsf/log?openagent&1270055001_sa4_2016_aust_shape.zip&1270.0.55.001&Data%20Cubes&C65BC89E549D1CA3CA257FED0013E074&0&July%202016&12.07.2016&Latest).

In [ ]:
import geopandas as gpd

sa4_gdf = gpd.read_file("data/sa4_2016_aust_shape/SA4_2016_AUST.shp")
sa4_gdf

## Libraries with Good GeoPandas support

Let's try to make a choropleth (spatial heat map) of wage data with a few different libraries.


In [ ]:
# utility function we'll need

def filter_and_merge_wage_data(sa4_gdf, jobs_df, date):
    """Filter to date and merge SA4 region data with wage data"""
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()][["SA4_CODE16", "geometry"]]
    filtered_df = jobs_df[jobs_df["Date"] == date]
    return sa4_gdf.merge(
        filtered_df,
        on="SA4_CODE16",
        validate="one_to_one",
    )

filter_and_merge_wage_data(sa4_gdf, jobs_df, "2020-01-11")

### GeoPandas & Matplotlib 

Like Pandas, Geopandas can produce plots using Matplotlib.


In [ ]:
%matplotlib widget

import contextily as cx
import matplotlib.pyplot as plt


def plot_wage_chloropleth_mpl(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    new_gdf = filter_and_merge_wage_data(sa4_gdf, jobs_df, date)
    fig, ax = plt.subplots()
   
    new_gdf.plot(
        ax=ax,
        edgecolor="black",
        column="Index",
        vmin=new_gdf["Index"].min(),
        vmax=new_gdf["Index"].max(),
        legend=True,
    ).set(title=f"Australian Jobs and Wages Index, {date}")

    # set the basemap tiles with contexily
    cx.add_basemap(ax, crs=new_gdf.crs.to_string(), source=cx.providers.CartoDB.Voyager)
    ax.axis("off")
    plt.show()


plot_wage_chloropleth_mpl(sa4_gdf, jobs_df, "2020-01-11")

We can use ipywidgets to make a simple UI to select months:

In [ ]:
from ipywidgets import interact

date_dropdowns = sorted(set(str(timestamp.date()) for timestamp in jobs_df["Date"]))

@interact(date=date_dropdowns)
def interactive_wages(date):
    plot_wage_chloropleth_mpl(sa4_gdf, jobs_df, date)

### Choropleth with GeoPandas & GeoPlot

In [ ]:
import contextily as cx
import geoplot as gplt


def plot_wage_chloropleth_geoplot(sa4_gdf, jobs_df, date):
    new_gdf = filter_and_merge_wage_data(sa4_gdf, jobs_df, date)

    geo_ax = gplt.choropleth(new_gdf, hue="Index", legend=True, linewidth=0.7)

    cx.add_basemap(
        geo_ax, crs=new_gdf.crs.to_string(), source=cx.providers.CartoDB.Voyager
    )
    plt.title(f"Australian Jobs and Wages Index, {date}")
    plt.show(geo_ax)


@interact(date=date_dropdowns)
def interactive_wages(date):
    plot_wage_chloropleth_geoplot(sa4_gdf, jobs_df, date)

### GeoPandas & GeoViews

Very slow :(

In [ ]:
import geoviews as gv

gv.extension("bokeh")

def plot_wage_chloropleth_geoviews(sa4_gdf, jobs_df, date):
    new_gdf = filter_and_merge_wage_data(sa4_gdf, jobs_df, date)
    return gv.Polygons(
        new_gdf,
        vdims=["Index"],
        label=f"Australian Jobs and Wages Index, {date}",
    ).opts(
        width=800,
        height=600,
        color="Index",
        colorbar=True,
    )


plot_wage_chloropleth_geoviews(sa4_gdf, jobs_df, "2020-01-11")

### GeoPandas & GeoViews

Very slow :(

In [ ]:
import hvplot.pandas


def plot_wage_chloropleth_hvplot(sa4_gdf, jobs_df, date):
    new_gdf = filter_and_merge_wage_data(sa4_gdf, jobs_df, date)
    return new_gdf.hvplot(geo=True, c="Index")

    
plot_wage_chloropleth_hvplot(sa4_gdf, jobs_df, "2020-01-04")

### GeoPandas & Folium

In [ ]:
import folium


def plot_wage_chloropleth_folium(sa4_gdf, jobs_df, date):
    """Plot a chloropleth map of jobs Index for a given date"""
    # remove records with empty geometry, and filter data to current month
    sa4_gdf = sa4_gdf[~sa4_gdf["geometry"].isnull()]
    filtered_df = jobs_df[jobs_df["Date"] == date]

    folium_map = folium.Map(location=[-22, 133], zoom_start=5)

    choropleth = folium.Choropleth(
        geo_data=sa4_gdf,
        data=filtered_df,
        columns=["SA4_NAME16", "Index"],
        key_on="feature.properties.SA4_NAME16",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Wage Index",
        highlight=True,
    )
    choropleth.geojson.add_child(
        folium.features.GeoJsonTooltip(["SA4_NAME16"], labels=False)
    )
    choropleth.add_to(folium_map)
    folium.LayerControl().add_to(folium_map)
    return folium_map


plot_wage_chloropleth_folium(sa4_gdf, jobs_df, "2020-01-04")

### GeoPandas & Ipyleaflet 

Supports GeoPandas for drawing polygons, but not for Choropleth :(

In [ ]:
from ipyleaflet import Choropleth, GeoData, Map, basemaps
from ipywidgets import Layout

py_map = Map(
    center=(-25.6091, 134.3619),
    zoom=3,
    basemap=basemaps.Esri.WorldTopoMap,
    scroll_wheel_zoom=True,
    layout=Layout(height="500px"),
)
sa4_layer = GeoData(geo_dataframe=sa4_gdf)
py_map.add_layer(sa4_layer)
py_map

To make a choropleth we have to do some wrangling to convert GeoPandas DataFrame into a GeoJSON dict

In [ ]:
import json


def geopandas_to_dict(sa4_gdf, jobs_df):
    """Create geojson dict from sa4 GeoPandas Dataframe"""
    # discard regions we don't have data for to make ipyleaflet's Choropleth happy
    target_regions = set(jobs_df["SA4_NAME16"].unique())
    sa4_gdf = sa4_gdf[sa4_gdf["SA4_NAME16"].isin(target_regions)]

    json_str = sa4_gdf.set_index("SA4_NAME16")["geometry"].to_json()
    return json.loads(json_str)


geojson = geopandas_to_dict(sa4_gdf, jobs_df)

date_values_map = {
    str(name.date()): dict(zip(jobs_df["SA4_NAME16"], jobs_df["Index"]))
    for name, df in jobs_df.groupby("Date")
}

In [ ]:
from ipyleaflet import Choropleth


def plot_wage_choropleth(geojson, choro_data):

    py_map = Map(
        center=(-25.6091, 134.3619),
        zoom=3,
        basemap=basemaps.Esri.WorldTopoMap,
        scroll_wheel_zoom=True,
        layout=Layout(height="500px"),
    )
    choro_layer = Choropleth(geo_data=geojson, choro_data=choro_data)
    py_map.add_layer(choro_layer)
    return py_map


plot_wage_choropleth(geojson, date_values_map["2020-01-11"])

## Viz Libraries that use GeoJSON

As far as I can tell, these all seem to require GeoJSON.

### Plotly

### HoloViews

### Altair

## What about big datasets?

DataShader